# Import Platform API

In [2]:
import requests
import json
from pprint import pprint
import boto3
import pandas as pd

# Initialization

In [3]:
output_location = 's3://feature-selection-process-bucket/'
##molex_data = “s3://kbs-analytics-molex-demand-forecasting/production/tuning/APS/monthly_agg_SKU_CUST_APS.csv”
molex_bucket = 'kbs-analytics-molex-demand-forecasting'
molex_file = "production/tuning/APS/monthly_agg_SKU_CUST_APS.csv"  

pred_bucket = 'prediction-services'
Baseurl = 'af862b932c04a40378f6ea79ad01b266-482589080.us-east-1.elb.amazonaws.com'
user = 'badrul'
password = "%IaY0lolDEOeQqsii$w9UO"


## Hyperparameter Tuning Using Prediction Services API - Single Model

In [6]:
#Baseurl = 'af2e270e8de2e415d9a909d9c27dc983-1063955246.us-east-1.elb.amazonaws.com:5000'
target_loc = 's3://' + pred_bucket + '/test_multi_target2.csv'
output_loc = 's3://'+ pred_bucket + '/parallel_ray_test'
params = {'model_training_runid': '1234-5674-7654-2345135',
 'runID': '1234-5674-7654-2345135',
 'model_name':['HOLTWINTERS'], 
 'model_parameters':{'HOLTWINTERS':
                     {'parameters':{'trend': ['add','mul'], 
                                    'damped': [True, False], 
                                    'seasonal_periods':3, 
                                    'seasonal':[None]},
                      "hyperparameters":{'smoothing_level':0.2}}
                      }, 
 'target_data_location': target_loc,
 'feature_data_location': None,
 's3_location':{
     'target_data': 's3://' + pred_bucket + '/test_multi_target.csv',
     'feature_data': None}, 
 'train_split_ratio':["1990-1-01","1990-1-26", "1990-1-29"], 
 'data_split_ratio':["1990-1-01","1990-1-26", "1990-1-29"], 
 'train_performance_metrics':['MAPE'],
 'test_performance_metrics':['MAPE'], 
 'parallelization_type':'Parallel', 
 'master_params':{'time_frequency':'M', 
                  'train_job_type':'fit_multiple',
                  'tuner':'ray_tuner', 
                  'data_config':'multi-targets', 
                  'file_type':'csv', 
                  'output_location':'s3://'+ pred_bucket + '/parallel_ray_test'}, 
          'train_job_type':'fit_multiple'}

response = requests.post("http://"+Baseurl+"/training", 
                         auth=(user, password),
                         data = json.dumps(params))    
response

<Response [200]>

## Hyperparameter Tuning Using Prediction Services API - MULTIPLE Models

In [7]:
##Baseurl='a34247cf2459f4cf7b3b89301b4a578c-616478844.us-east-1.elb.amazonaws.com:5000'
##Baseurl = 'af2e270e8de2e415d9a909d9c27dc983-1063955246.us-east-1.elb.amazonaws.com:5000'
target_loc = 's3://' + pred_bucket + '/test_multi_target.csv'
output_loc = 's3://'+ pred_bucket + '/parallel_ray_test'
params = {'model_training_runid': '1234-5674-7654-2345134',
 'runID': '1234-5674-7654-2345134',
 'model_name':['ARIMA', 'HOLTWINTERS', 'PROPHET'], 
 'model_parameters':{'ARIMA':
                     {'parameters':{'p':[1,7], 'd':[1,2], 'q':[1,2]},
                      'hyperparameters':{'disp':0}},
                     'HOLTWINTERS':
                     {'parameters':{'trend': ['add','mul'], 
                                    'damped': [True, False], 
                                    'seasonal_periods':3, 
                                    'seasonal':[None]},
                      "hyperparameters":{'smoothing_level':0.2}},
                      'PROPHET':
                     {'parameters':{'growth':['linear'], 
                                    'changepoints': [None], 
                                    'n_changepoints': [20, 22], 
                                    'changepoint_range': [0.8, 0.9],
                     'changepoint_prior_scale': [0.05, 0.1], 
                                    'yearly_seasonality': ['auto'], 
                                    'weekly_seasonality': ['auto'], 
                                    'daily_seasonality': ['auto'],
                         'holidays': [None], 
                                    'seasonality_mode': ['additive','multiplicative'], 
                                    'seasonality_prior_scale': [10.0], 
                                    'holidays_prior_scale': [10.0],
                         'interval_width': [0.8], 
                                    'uncertainty_samples': [1000]
                                   }}}, 
 'target_data_location': target_loc,
 'feature_data_location': None,
 's3_location':{
     'target_data': 's3://' + pred_bucket + '/test_multi_target.csv',
     'feature_data': None}, 
 'train_split_ratio':["1990-1-01","1990-1-26", "1990-1-29"], 
 'data_split_ratio':["1990-1-01","1990-1-26", "1990-1-29"], 
 'train_performance_metrics':['MAPE'],
 'test_performance_metrics':['MAPE'], 
 'parallelization_type':'Parallel', 
 'master_params':{'time_frequency':'M', 
                  'train_job_type':'fit_multiple',
                  'tuner':'ray_tuner', 
                  'data_config':'multi-targets', 
                  'file_type':'csv', 
                  'output_location':'s3://'+ pred_bucket + '/parallel_ray_test'}, 
          'train_job_type':'fit_multiple'}
Baseurl = 'af862b932c04a40378f6ea79ad01b266-482589080.us-east-1.elb.amazonaws.com'
user = 'badrul'
password = "%IaY0lolDEOeQqsii$w9UO"
 
response = requests.post("http://"+Baseurl+"/training", 
                         auth=(user, password),
                         data = json.dumps(params))    
response

<Response [200]>

In [8]:
print(response)
print(target_loc)
print(output_loc)
print(Baseurl)
model_train_run_id = '1'
print(model_train_run_id)
##print(json.dumps(params))

<Response [200]>
s3://prediction-services/test_multi_target.csv
s3://prediction-services/parallel_ray_test
af862b932c04a40378f6ea79ad01b266-482589080.us-east-1.elb.amazonaws.com
1


In [9]:
type(response)
print(model_train_run_id)
status_response = requests.get("http://"+Baseurl+"/trainings/"+model_train_run_id,
                              auth=(user,password))    
print(status_response)

1
<Response [200]>


# Prediction using Platform API

In [ ]:
prediction_config = {"forecast_steps":3,
                    'score_file_location': '',
                     's3_location':{
     'target_data': 's3://' + pred_bucket + '/test_multi_target.csv',
     'feature_data': None}}
target_data_config = "edd0689f-b516-4770-ba82-7baa3193e69a"
feature_data_config = None
model_training_run_id = model_train_run_id

In [ ]:
response = requests.post("http://"+Baseurl+"/run_score", data = json.dumps(prediction_config))    

predict_response = ps_apiv1.create_model_predict(name, description, product_name, model_training_run_id, target_data_config, feature_data_config, prediction_config)
predict_response